In [1]:
import torch
import pandas as pd
from torch.utils.data import Dataset, TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import random
import os
import csv
import numpy as np
from tqdm import tqdm
import dask.dataframe as dd

In [2]:
# Choose device for torch computing
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
# Change working directory to labels
work_dir = "C:/file_lists_with_labels_ff_estimator"
os.chdir(work_dir)

# Reads labels into dict of (filename: ff_value)
training_data_labels = {}
with open("training.txt") as f:
    for line in f:
        key, val = line.split()
        training_data_labels[key] = float(val)
        
test_data_labels = {}
with open("test.txt") as f:
    for line in f:
        key, val = line.split()
        test_data_labels[key] = float(val)

In [4]:
# Change directory to data
os.chdir("C:\\rf_without_tgc")

# Iterate thru data to create lists of tensors
training_data = []
training_labels = []
for file in tqdm(os.listdir()):
    if file not in training_data_labels:
        continue
        
    file_data = pd.read_csv(file, header=None).T
    
    x_tensor = file_data.to_numpy().astype(np.float32)
    y_tensor = float(training_data_labels[file])
    
    for x in x_tensor:
        training_data.append(x)
        training_labels.append(y_tensor)
    
test_data = []
test_labels = []
for file in tqdm(os.listdir()):
    if file not in test_data_labels:
        continue
        
    file_data = pd.read_csv(file, header=None).T
    
    x_tensor = file_data.to_numpy().astype(np.float32)
    y_tensor = float(test_data_labels[file])

    for x in x_tensor:
        test_data.append(x)
        test_labels.append(y_tensor)

100%|██████████████████████████████████████████████████████████████████████████████| 2040/2040 [00:47<00:00, 42.75it/s]


In [5]:
# # Create class for DataLoader compatability
# class Data():
#     def __init__(self, x, y):
#         self.x = x
#         self.y = y
    
#     def __len__(self):
#         return len(self.x)

#     def __getitem__(self, idx):
#         X = self.x[idx]
#         y =  self.y[idx]

#         return X, y

training_data = np.array(training_data)
training_labels = np.array(training_labels)

test_data = np.array(test_data)
test_labels = np.array(test_labels)

print(training_data.shape)
print(test_data.shape)
    
# training_data = np.swapaxes(training_data, 1, 2)
# test_data = np.swapaxes(test_data, 1, 2)

# Create data tensors
training_data = torch.Tensor(training_data)
training_labels = torch.Tensor(training_labels)

test_data = torch.Tensor(test_data)
test_labels = torch.Tensor(test_labels)

# training_data = (training_data - torch.mean(training_data)) / torch.std(training_data)
# test_data = (test_data - torch.mean(test_data)) / torch.std(test_data)

train_dataset = TensorDataset(training_data, training_labels)
test_dataset = TensorDataset(test_data, test_labels)

# # Load tensors into class for torch DataLoaders
# train_data = Data(training_data, training_labels)
# test_data = Data(test_data, test_labels)

(261120, 1024)
(261120, 1024)


In [6]:
# DataLoader Parameters
loader_params = {
    "batch_size":  32, 
    "shuffle":     True,
    "num_workers": 0
}

# Create DataLoader for training data
loader = DataLoader(train_dataset, **loader_params)

In [7]:
import torch.nn as nn

pool = 4

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 8, 8, 1, padding='same'),
            nn.Tanh(),
            nn.MaxPool1d(pool, stride=pool, padding=0)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv1d(8, 8, 4, 1, padding='same'),
            nn.Tanh(),
            nn.MaxPool1d(pool, stride=pool, padding=0)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv1d(8, 16, 4, 1, padding='same'),
            nn.Tanh(),
            nn.MaxPool1d(pool, stride=pool, padding=0)
        )
        
        self.fc1 = nn.Sequential(
            nn.Linear(256, 32),
            nn.Tanh()
        )
        
        self.fc2 = nn.Linear(32, 1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

# import torch.nn as nn

# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
        
#         self.conv1 = nn.Sequential(
#             nn.Conv2d(1, 8, 16, 1, 'same'),
#             nn.Tanh(),
#             nn.MaxPool2d(2, stride=2, padding=0)
#         )
        
#         self.conv2 = nn.Sequential(
#             nn.Conv2d(8, 8, 8, 1, padding='same'),
#             nn.Tanh(),
#             nn.MaxPool2d(2, stride=2, padding=0)
#         )
        
#         self.conv3 = nn.Sequential(
#             nn.Conv2d(8, 16, 8, 1, padding='same'),
#             nn.Tanh(),
#             nn.MaxPool2d(2, stride=2, padding=0)
#         )
        
#         self.fc1 = nn.Sequential(
#             nn.Flatten(),
#             nn.Linear(65024, 32),
#             nn.Tanh()
#         )
        
#         self.fc2 = nn.Linear(32, 2)
    
#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.conv2(x)
#         x = self.conv3(x)
#         x = self.fc1(x)
#         x = self.fc2(x)
#         return x

In [8]:
# Initialise net and push to computing device
cnn = CNN()
cnn.to(device)

CNN(
  (conv1): Sequential(
    (0): Conv1d(1, 8, kernel_size=(8,), stride=(1,), padding=same)
    (1): Tanh()
    (2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv1d(8, 8, kernel_size=(4,), stride=(1,), padding=same)
    (1): Tanh()
    (2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv1d(8, 16, kernel_size=(4,), stride=(1,), padding=same)
    (1): Tanh()
    (2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Sequential(
    (0): Linear(in_features=256, out_features=32, bias=True)
    (1): Tanh()
  )
  (fc2): Linear(in_features=32, out_features=1, bias=True)
)

In [9]:
# Define loss function
loss_func = nn.MSELoss()

In [10]:
from torch import optim

# Define optimiser and learning rate
optimiser = optim.Adagrad(cnn.parameters(), lr=.005)

In [11]:
from torch.autograd import Variable
import time

num_epochs = 50

def train(num_epochs, cnn, loader):
    cnn.train()
    
    accum = 8
    
    total_step = len(loader)
    
    for epoch in range(num_epochs):        
        for i, (signal, label) in enumerate(loader):
            # Send data to device
            signal, label = signal.to(device), label.to(device)
            signal = torch.unsqueeze(signal, 1)
            
            with torch.set_grad_enabled(True):
                # Pass inputs through net and obtain outputs
                output = cnn(signal)

                # Cast type to float and flatten results
                output = output.to(torch.float)
                output = torch.squeeze(output)
                label = label.to(torch.float)
                
#                 print(output.shape)
#                 print(label.shape)
            
                # Call loss function on net outputs
                loss = loss_func(output, label)

    #             a = list(cnn.parameters())[0].clone()

                # Computes gradients for weights
                loss.backward()            

                if ((i + 1) % accum == 0) or (i + 1 == len(loader)):
                    # Apply gradients using optimiser policy
                    optimiser.step()

                    # Zero network gradients
                    optimiser.zero_grad()

#                 print(list(cnn.parameters())[0].grad)
    #             b = list(cnn.parameters())[0].clone()

    #             print(torch.equal(a.data, b.data))

            
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, loss.item()))

start = time.time()

train(num_epochs, cnn, loader)

end = time.time()
print(end - start)

C:\Users\willi\Anaconda3\lib\site-packages\torch\nn\modules\conv.py:298: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\Convolution.cpp:744.)
  return F.conv1d(input, weight, bias, self.stride,


Epoch [1/50], Loss: 63.0360
Epoch [2/50], Loss: 41.7159
Epoch [3/50], Loss: 89.9523
Epoch [4/50], Loss: 48.8297
Epoch [5/50], Loss: 21.8218
Epoch [6/50], Loss: 36.5319
Epoch [7/50], Loss: 27.3808
Epoch [8/50], Loss: 13.1044
Epoch [9/50], Loss: 33.9479
Epoch [10/50], Loss: 32.5101
Epoch [11/50], Loss: 18.9605
Epoch [12/50], Loss: 29.7588
Epoch [13/50], Loss: 26.5330
Epoch [14/50], Loss: 12.3110
Epoch [15/50], Loss: 23.9676
Epoch [16/50], Loss: 37.0451
Epoch [17/50], Loss: 30.7373
Epoch [18/50], Loss: 25.9636
Epoch [19/50], Loss: 19.2997
Epoch [20/50], Loss: 20.4021
Epoch [21/50], Loss: 14.8886
Epoch [22/50], Loss: 17.3342
Epoch [23/50], Loss: 32.8667
Epoch [24/50], Loss: 31.1784
Epoch [25/50], Loss: 33.4727
Epoch [26/50], Loss: 33.5238
Epoch [27/50], Loss: 16.9639
Epoch [28/50], Loss: 15.8366
Epoch [29/50], Loss: 26.0659
Epoch [30/50], Loss: 30.1807
Epoch [31/50], Loss: 16.9325
Epoch [32/50], Loss: 15.8137
Epoch [33/50], Loss: 30.1081
Epoch [34/50], Loss: 27.5613
Epoch [35/50], Loss: 18

In [12]:
import scipy

# DataLoader Parameters
loader_params = {
    "batch_size":  1, 
    "shuffle":     False,
    "num_workers": 0
}

# Create test DataLoader
test_loader = DataLoader(test_dataset, **loader_params)

def test():
    cnn.eval()
    
    # Initialise arrays and dict
    predictions = np.array([])
    labels = np.array([])
    averaged_dict = {}
    
    with torch.no_grad():
        for i, (signal, label) in tqdm(enumerate(test_loader)):
            # Send input to device
            signal = torch.Tensor(signal).to(device)
            signal = torch.unsqueeze(signal, 1)
             
            # Get output of net, append to lists
            output = torch.squeeze(cnn(signal)).cpu().detach().numpy()
            predictions = np.append(predictions, output)            
            labels = np.append(labels, label)
        
#             print(output, label)
        
        for i in range(len(labels)):
            if labels[i] not in averaged_dict:
                averaged_dict[labels[i]] = [predictions[i]]
            else:
                averaged_dict[labels[i]].append(predictions[i])
            
    for i in averaged_dict:
        averaged_dict[i] = np.mean(averaged_dict[i])
    
    averaged_predictions = []
    ordered_labels = []
        
    for i in averaged_dict:
        ordered_labels.append(i)
        averaged_predictions.append(averaged_dict[i])
    
    print(predictions)
    print(labels)
    print(labels.shape)
    
#     r = scipy.stats.pearsonr(predictions, labels)
    
    r = scipy.stats.pearsonr(averaged_predictions, ordered_labels)
    
    print('Pearson r of the model is %.2f' % r[0])
    
test()

261120it [06:19, 688.76it/s] 


[10.76714325 10.56886196  9.62124825 ... 17.52271843 14.27943325
 18.94706917]
[ 0.546314    0.546314    0.546314   ... 34.99910736 34.99910736
 34.99910736]
(261120,)
Pearson r of the model is 0.82
